In [1]:
import os
import pandas as pd

In [12]:
Phone1Files = []
Phone2Files = []

for root, dirs, files in os.walk("./data"):
    for name in files:
        fpath = os.path.join(root, name)
        if name.find("Phone_1") != -1:
            Phone1Files.append(fpath)
        else:
            Phone2Files.append(fpath)

In [63]:
phone1df = pd.read_csv(Phone1Files[1])
phone1df.drop(phone1df.tail(8).index, inplace=True) #Removing aggregate Statistics

/tmp/ipykernel_7048/2561763531.py:1: DtypeWarning: Columns (186,189,190,197,223,236,240,243,244,251,276,280,283,284,291,314,317,318,325,338,351,355,358,359,366,388) have mixed types. Specify dtype option on import or set low_memory=False.
  phone1df = pd.read_csv(Phone1Files[1])


In [64]:
phone1df["TIME_STAMP"] = pd.to_datetime(phone1df["TIME_STAMP"])
phone1df = phone1df.sort_values("TIME_STAMP")
phone1df["Experiment"] = 0
phone1df['Experiment'] = (phone1df['TIME_STAMP']
                   .transform(lambda x: x.diff().gt('1Sec').cumsum())
                )
phone1df["Experiment"].max()

29